In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from google.colab import files
from PIL import Image
import numpy as np

from tqdm.notebook import tqdm

from IPython.display import display

In [ ]:
base_model = keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [ ]:
names = ['mixed3', 'mixed5']

outputs = [base_model.get_layer(layer).output for layer in names]
model = keras.Model(inputs=base_model.input, outputs=outputs)

In [ ]:
def calculate_loss(img, model):
  img = tf.expand_dims(img, axis=0)

  activations = model(img)
  losses = [tf.reduce_mean(activation) for activation in activations]

  return tf.reduce_sum(losses)

In [ ]:
def upload_image():
  image = files.upload()
  file_name = list(image.keys())[0]

  print()
  print(f'uploaded file: {file_name}')

  image = Image.open(file_name)
  image = np.array(image)
  image = tf.convert_to_tensor(image)
  image = tf.cast(image, tf.float32)

  return image

In [ ]:
def step(image, model, count, step_size):
  for _ in range(count):
    with tf.GradientTape() as tape:
      tape.watch(image)

      loss = calculate_loss(image, model)
    
    gradients = tape.gradient(loss, image)
    gradients /= tf.math.reduce_std(gradients) + 1e-8 

    image = image + gradients * step_size
    image = tf.clip_by_value(image, -1, 1)

  return image, loss

In [ ]:
def generate(image, model, count, step_size):
  image = tf.keras.applications.inception_v3.preprocess_input(image)
  for i in tqdm(range(1, count + 1)):
    image, loss = step(image, model, 100, tf.constant(step_size))

    print (f"Step {i}, loss {loss}")

  return image

In [ ]:
def convert_image(image):
  image = 255. * (image + 1.) / 2.
  image = tf.cast(image, tf.uint8)

  return image

In [ ]:
count = 10
step_size = 1e-2

In [ ]:
image = upload_image()

In [ ]:
final_image = generate(image, model, count, step_size)

In [ ]:
converted_image = convert_image(final_image)

In [ ]:
converted_image = Image.fromarray(converted_image.numpy())
display(converted_image)

In [ ]:
converted_image.save('result.jpg')
files.download('result.jpg')